## Grant Planning package with People and Money

### Installation
In addition to jupyter notebook, your machine should have these:
```
pip3 install matplotlib
pip3 install pandas
pip3 install openpyxl
pip3 install xlrd
```

### Usage
Change directory to where the notebooks (.ipynb) files are and 
```
jupyter notebook
```
The code will expect to find an Excel file about projects, how much is 
awarded and spent in various accounting categories. The code will also 
expect an Excel file of transactions, about what has been spent on which grant 
in various categories.

There are two Excel files that come from the People and Money system:
`project.xls` and `transaction.xlsx`
 
There are four supporting concepts to understand. Three of these rely
on additional JSON files. After running the following code snippets, 
you can clear the cell content by selecting the cell then from the menu
'Cell/Current Outputs/Clear'

### Settings
Here are directory and file names that will be used

In [1]:
import settings
print('mygrantsjson is            ', settings.MYGRANTS)
print('projects spreadsheet is    ', settings.PROJECTS, ' dated ', settings.PROJECTS_DATE)
print('people.json is             ', settings.PEOPLE)
print('assign.json is             ', settings.ASSIGN)
print('transactions spreadsheet is', settings.TRANSACTIONS)
print('This run is from', settings.RUN_START, 'to', settings.RUN_END)

mygrantsjson is             toydata/mygrants.json
projects spreadsheet is     toydata/projects.xlsx  dated  Aug-22
people.json is              toydata/people.json
assign.json is              toydata/assignFTE.json
transactions spreadsheet is toydata/transactions.xlsx
This run is from Sep-22 to Nov-24


---
### Run
This code covers an interval of months, from a month in the past
that corresponds to the same month of the `project.xls`, to a month
in the future. Actual expenditure is from the run start until the 
date of the `transactions.xlsx` file.

In [2]:
import util
run = util.run(settings.RUN_START, settings.RUN_END)
run.print()

Run is from Sep-22 to Nov-24 (26 months)


---
### People
A JSON file contains last names of the personnel, with 
their [spine points](https://www.ed.ac.uk/human-resources/pay-reward/pay/pay-scales) and salary. There is a multiplier to convert 
salary for the person into cost for the grant. There is a python
module people.py. You can run the cell below to see the results.

In [3]:
import people
pe = people.people(settings.PEOPLE)
print(pe.all_names())
# Restrict attention to just these
#pe.set_names(['Bloggs'])

['Bloggs', 'Jones']


---
### Grants
Each grant is a combination of a hand-made file with short names and colours, 
together with the projects Excel file from P&M. The two are combined into a 
single file grants.json that is used for the plots and tables.

In [4]:
import grants
gr = grants.grants(settings.MYGRANTS)
gr.from_projects  (settings.PROJECTS, settings.PROJECTS_DATE)
print(gr.all_names())
# Restrict attention to just these
#gr.set_names(['Venice'])

['ToyGaia', 'Venice']


This file `mygrants.json` is human editable, just a choice of short names and colours really. 
It is meant to be supplemented with information from the `projects.xls`. 
Sometimes the P&M reported end time for the grant is not appropriate, and it can be 
specificed in this file to override the spreadsheet from P&M.

In [5]:
gr.print()

As of beginning of Aug-22
  P&M project  10446744_10446748 ToyGaia ToyGaia Data Flow System: 2021-2024 (Edinburgh 2021/2022 Element)
   Start 31-Mar-2022, End Dec-23 (20 months)
   Awarded     Spent    Category
    100000      20000   Salary
     10000       2000   Travel
     30000       5000   Equipment
     30000       5000   Consumables

  P&M project  11111995_11111999 Venice Reviving the Empire of Venice
   Start 31-Jun-  22, End 31-Oct-2024 (28 months)
   Awarded     Spent    Category
    200000      10000   Salary
     30000       5000   Travel
     30000       5000   Equipment
     30000       5000   Consumables



Grants are assigned colours in the `mygrants.json` file, 
which can be seen by running the following cell:

In [6]:
from IPython.display import display, HTML
out = gr.colour_chart()
display(HTML(out))

---
### Assign
This is the forecasting part of the system, where FTE and salary are 
combined to see the effect on the grant. There is no forecasting of non-salary expenses.

People are assigned to grants through records like:
["Davidson",    "Gaia21To22",    "Jan-22",    0.5],
which says the person Davidson will work 50% on the grant Gaia21To22, 
starting Jan-22. The notebook Assign_Tool.ipynb shows who is working 
on which grant month by month, together with the sum of all the 
FTE percentages, which should add to 1.0 for a full-time employee.
Running the following cell will be helpful in editing the assignFTE.json file.

In [7]:
import assign
af = assign.assign(settings.ASSIGN, gr, pe, run)
af.print()

Bloggs
   Sep-22 (0.00): 
   Oct-22 (1.00): 1.00 on    ToyGaia| 
   Nov-22 (1.00): 1.00 on    ToyGaia| 
   Dec-22 (1.00): 1.00 on    ToyGaia| 
   Jan-23 (1.00): 1.00 on    ToyGaia| 
   Feb-23 (1.00): 1.00 on    ToyGaia| 
   Mar-23 (1.00): 1.00 on    ToyGaia| 
   Apr-23 (1.00): 1.00 on    ToyGaia| 
   May-23 (1.00): 1.00 on    ToyGaia| 
   Jun-23 (1.00): 1.00 on    ToyGaia| 
   Jul-23 (0.00): 
   Aug-23 (0.00): 
   Sep-23 (0.00): 
   Oct-23 (0.00): 
   Nov-23 (0.00): 
   Dec-23 (0.00): 
   Jan-24 (0.00): 
   Feb-24 (0.00): 
   Mar-24 (0.00): 
   Apr-24 (0.00): 
   May-24 (0.00): 
   Jun-24 (0.00): 
   Jul-24 (0.00): 
   Aug-24 (0.00): 
   Sep-24 (0.00): 
   Oct-24 (0.00): 
Jones
   Sep-22 (0.00): 
   Oct-22 (0.00): 
   Nov-22 (0.00): 
   Dec-22 (0.00): 
   Jan-23 (0.00): 
   Feb-23 (0.00): 
   Mar-23 (0.00): 
   Apr-23 (0.50): 0.50 on    ToyGaia| 
   May-23 (0.50): 0.50 on    ToyGaia| 
   Jun-23 (0.50): 0.50 on    ToyGaia| 
   Jul-23 (1.00): 0.50 on    ToyGaia| 0.50 on     Venice| 
   A

---
### Non-salary expenses
Each grant may be awarded money in several expense catagories. 
However some of these are ignored by this system, 
since the PI has no control over their expenditure. Some of the controllable
expense categories are lumped together. This can be seen in the code `util.py`,
where currently we have:

In [8]:
import util
print('All considered "Salary":',      util.category_salary)
print()
print('All considered "Consumables":', util.category_consumables)
print()
print('All considered "Travel":',      util.category_travel)
print()
print('All considered "Equipment":',   util.category_equipment)
print()
print('And these are ignored:',        util.category_ignore)

All considered "Salary": ['Research Investigator', 'Research Assistant']

All considered "Consumables": ['Consumables - Research Other Costs', 'Consumables - IT', 'Consumables - Telephone and Communication', 'Consumables - Printing Postage and Stationery', 'Consumables - Research Consumables', 'Staff Conf Course and Seminar Fees', 'Other Directly Incurred']

All considered "Travel": ['Travel and Subsistence']

All considered "Equipment": ['Equipment']

And these are ignored: ['nan', 'Total Award', 'Grant Income', 'Financial Resources', 'Directly Allocated - Co-Principal Investigator Staff', 'Directly Allocated - Estates Costs', 'Directly Allocated - Infrastructure Technician Costs', 'Directly Allocated - Principal Investigator Staff', 'Directly Allocated - Co-Principal Investigator Staff', 'Directly Allocated Research Administration Staff', 'Indirect Costs']


---
### Notebook Grants.ipynb
Each grant is shown in order of the short name, with the amount awarded and spent, 
in each of the categories. Note: we always assume the start of the run is the same as 
the date of the `project.xls` file.

Four plots correspond to the four expense categories, showing actual expense from the
`transactions.xlsx` file, as well as a grey line of constant slope from what is spent at the 
beginning of the run to the assumption of full spend at the end of the grant. 
The Salary plot also shows forecast expense to the end of the run.

The table 'Forecast Salary' is from the `assign.json` file,
together with cost summed ove people, cumulative spend on salary,
and remaining balance. The table 'Salary Spending' is similar, but with actual expenses
from the `transactions.xlsx` file. The table 'Category Spending' shows 
expenditure in the four categories.

### Notebook People.ipynb
Each person is shown with two plots. The left is the forecast of their
FTE on the various grants, and the right is their actual costs
charged to different grants, converted to FTE.